In [1]:
import json
from numpyencoder import NumpyEncoder
import sys
sys.path.insert(0, '../../')
from util.evaluator import *

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [2]:
def run_cfmss(json_path):
    with open(json_path) as f:
        cfmss_json = json.load(f)

    cfmss_json.keys()
    d = len(cfmss_json['data'][0][0])
    cfs_list = cfmss_json['cf']
    _cfs_list = cfmss_json['cf2']
    diversity_list = []
    diversity2_list = []

    count_diversity_list = []
    count_diversity2_list = []

    for cfs, _cfs in zip(cfs_list, _cfs_list):
        cfs = [item['cf'] for item in cfs]
        cfs = np.reshape(cfs, (-1, d))
        # print(cfs.shape)
        diversity = evaluator.diversity(cfs)
        count_diversity = evaluator.count_diversity(cfs)

        _cfs = [item for item in _cfs]
        _cfs = np.reshape(_cfs, (-1, d))
        print(_cfs.shape)
        diversity2 = evaluator.diversity(_cfs)
        count_diversity2 = evaluator.count_diversity(_cfs)

        diversity_list.append(diversity)
        diversity2_list.append(diversity2)
        count_diversity_list.append(count_diversity)
        count_diversity2_list.append(count_diversity2)

    print(cfmss_json['diversity'])
    print(diversity_list)

    print(cfmss_json['diversity2'])
    print(diversity2_list)

    # del cfmss_json['diversity']
    # del cfmss_json['diversity2']
    # del cfmss_json['count_diversity']
    # del cfmss_json

    cfmss_json['diversity'] = diversity_list
    cfmss_json['diversity2'] = diversity2_list
    cfmss_json['count_diversity'] = count_diversity_list
    cfmss_json['count_diversity2'] = count_diversity2_list
    with open(json_path, "w") as f:
        json.dump(cfmss_json, f, cls = NumpyEncoder)

def run(json_path):
    with open(json_path) as f:
        cfmss_json = json.load(f)

    cfmss_json.keys()
    d = len(cfmss_json['data'][0][0])
    cfs_lists = cfmss_json['cf']
    diversity_list = []
    diversity2_list = []

    count_diversity_list = []
    count_diversity2_list = []

    for cfs_list in cfs_lists:

        cfs = [_tmp_result['cf'] for _tmp_result in cfs_list]
        cfs = np.reshape(cfs, (-1, d))
        diversity = evaluator.diversity(cfs)
        count_diversity = evaluator.count_diversity(cfs)

        _cfs = [_tmp_result['cf2'] for _tmp_result in cfs_list]
        _cfs = np.reshape(_cfs, (-1, d))
        diversity2 = evaluator.diversity(_cfs)
        count_diversity2 = evaluator.count_diversity(_cfs)

        diversity_list.append(diversity)
        diversity2_list.append(diversity2)
        count_diversity_list.append(count_diversity)
        count_diversity2_list.append(count_diversity2)

    print(cfmss_json['diversity'])
    print(diversity_list)

    print(cfmss_json['diversity2'])
    print(diversity2_list)

    # del cfmss_json['diversity']
    # del cfmss_json['diversity2']

    cfmss_json['diversity'] = diversity_list
    cfmss_json['diversity2'] = diversity2_list
    cfmss_json['count_diversity'] = count_diversity_list
    cfmss_json['count_diversity2'] = count_diversity2_list
    with open(json_path, "w") as f:
        json.dump(cfmss_json, f, cls = NumpyEncoder)

# hepatitis

In [3]:
from hepatitis_update import *

data = pd.read_csv("../../data/Hepatitis/HepatitisC_dataset_processed.csv")
standard_sc = StandardScaler()

X = data.drop(['Category'],axis=1)
y = data["Category"]
train_x, test_x, train_y, test_y = train_test_split(X,y,test_size=0.3,random_state=42,stratify=y)

train_x=standard_sc.fit_transform(train_x).astype(np.float32)
test_x=standard_sc.transform(test_x).astype(np.float32)

train_y = train_y.to_numpy().astype(np.int64)
test_y = test_y.to_numpy().astype(np.int64)

model_path = '../../train/hepatitis/Hepatitis_model_simple.pt'
baseline_model = create_model_from_torch(model_path)
model_path1 = '../../train/hepatitis/Hepatitis_model_simple_v1.pt'
baseline_model1 = create_model_from_torch1(model_path1)

# obtain true positive set of test set
idx = np.where(test_y == 0)[0]
pred_y = np.argmax(baseline_model.predict(test_x), axis = 1)
idx1 = np.where(pred_y == 0)[0]
pred_y_ = np.argmax(baseline_model1.predict(test_x), axis = 1)
idx1_ = np.where(pred_y_ == 0)[0]
tn_idx = set(idx).intersection(idx1)
tn_idx = set(tn_idx).intersection(idx1_)
abnormal_test = test_x[list(tn_idx)]

# obtain true negative set of train set
idx2 = np.where(train_y == 1)[0]
pred_ty = baseline_model.predict(train_x).round()
pred_ty = np.argmax(baseline_model.predict(train_x), axis = 1)
idx3 = np.where(pred_ty == 1)[0]
tp_idx = set(idx2).intersection(idx3)
normal_test = train_x[list(tp_idx)]

# initialize the evaluator
evaluator = Evaluator(train_x, normal_test)

14/14 [==============================] - 0s 2ms/step


In [4]:
hepatitis_update_path = 'hepatitis_sns_update.json'
run(hepatitis_update_path)

[0.08431685039902644, 0.0019210262688733938, 0.04578612149787101, 0.011044469531564438, 0.014384940322352192, 0.007140411605324104, -1, -1, -1, 0.01864960269985505, -1, 0.06062073405112304, 0.0030632713968893554, 0.06105774374641541, -1, 0.012314295599675597, -1, 0.011715493946706169, 0.02760479154056929]
[0.8431685039902644, 0.019210262688733937, 0.4578612149787101, 0.11044469531564438, 0.14384940322352194, 0.07140411605324104, -1, -1, -1, 0.1864960269985505, -1, 0.6062073405112304, 0.030632713968893555, 0.6105774374641542, -1, 0.12314295599675597, -1, 0.11715493946706168, 0.2760479154056929]
[0.02040274039272608, 0.0028074460020895754, 0.02244893302169406, 0.1285831598844605, 0.058295370484302625, 0.02284116219888723, -1, -1, -1, 0.01641231846751685, -1, 0.01098755771808344, 0.0027391501099994406, 0.07122444795113435, -1, 0.06628113984597021, -1, 0.06534181234935872, 0.025036454765121938]
[0.2040274039272608, 0.028074460020895753, 0.22448933021694062, 1.2858315988446052, 0.5829537048

# Syn

In [5]:
from synthetic_update import *

dataset = pd.read_csv("../../data/synthetic/synthetic_data_simple.csv")
dataset = dataset.to_numpy().astype(np.float32)
lens = len(dataset)
train, test = dataset[0:int(lens * 0.7), ], dataset[int(lens*0.7):, ]
train_x, train_y = train[:, 0:4], train[:, 4]
test_x, test_y = test[:, 0:4], test[:, 4]
train_y = train_y.astype(np.int64)
test_y = test_y.astype(np.int64)

model_path = '../../train/synthetic/synthetic_model_simple.pt'
baseline_model = create_model_from_torch(model_path)
model_path1 = '../../train/synthetic/synthetic_model_simple_v1.pt'
baseline_model1 = create_model_from_torch1(model_path1)

# obtain true positive set of test set
idx = np.where(test_y == 0)[0]
pred_y = np.argmax(baseline_model.predict(test_x), axis = 1)
idx1 = np.where(pred_y == 0)[0]
pred_y_ = np.argmax(baseline_model1.predict(test_x), axis = 1)
idx1_ = np.where(pred_y_ == 0)[0]
tn_idx = set(idx).intersection(idx1)
tn_idx = set(tn_idx).intersection(idx1_)
abnormal_test = test_x[list(tn_idx)]

# obtain true negative set of train set
idx2 = np.where(train_y == 1)[0]
pred_ty = baseline_model.predict(train_x).round()
pred_ty = np.argmax(baseline_model.predict(train_x), axis = 1)
idx3 = np.where(pred_ty == 1)[0]
tp_idx = set(idx2).intersection(idx3)
normal_test = train_x[list(tp_idx)]

# initialize the evaluator
evaluator = Evaluator(train_x, normal_test)

438/438 [==============================] - 0s 677us/step


In [6]:
syn_update_path = 'synthetic_sns_update.json'
run(syn_update_path)

[0.20055137417506744, 0.4873646282772353, 0.6029925200010874, 0.14821002672604955, 0.4311703732455423, 0.4769879432102891, 0.3909128198065792, 0.09168525099266844, 0.31210940951452254, 0.50059076866706, 0.17996547419633005, 0.8072037949794747, 0.38891602719259194, 0.3839338213696714, 0.190779035862754, 0.058141523784573046, 0.5922184927478553, 0.09186230457691247, 0.4100475688097242, 1.0128313082129428, 0.3652907324541683, 0.12087050417270451, 0.7019406191289224, 0.09319741314523948, 1.038798670033134, 0.24954606609084506, 0.1691697314459759, 0.3765839870630951, 0.363766731062393, 0.006205825525025688, 1.213301093465902, 0.04390700105900791, 0.16663680593743901, 0.37436584968484987, 0.30399645584842244, 0.7401854438372486, 0.18971245748493848, 0.16151204597680852, 1.1602425917650088, 0.5360762401863461, 0.17155873111270403, 0.2044064190430258, 0.14635024958870346, 0.21655565321573333, 1.1522246918498027, 1.7316034599325403, 0.2713770292350457, 0.6803549751942711, 0.39162925322353104, 0

# thyroid

In [7]:
from thyroid_update import *

dataset = pd.read_csv("../../data/thyroid/thyroid_dataset.csv")
dataset = dataset.to_numpy().astype(np.float32)
lens = len(dataset)
train, test = dataset[0:int(lens * 0.7), ], dataset[int(lens*0.7):, ]
train_x, train_y = train[:, 0:4], train[:, 4]
test_x, test_y = test[:, 0:4], test[:, 4]

scaler = StandardScaler()
train_x = scaler.fit_transform(train_x)
test_x = scaler.transform(test_x)
train_y = train_y.astype(np.int64)
test_y = test_y.astype(np.int64)

model_path = '../../train/thyroid/thyroid_data.pt'
baseline_model = create_model_from_torch(model_path)
model_path1 = '../../train/thyroid/thyroid_data_new.pt'
baseline_model1 = create_model_from_torch1(model_path1)

# obtain true positive set of test set
idx = np.where(test_y == 0)[0]
pred_y = np.argmax(baseline_model.predict(test_x), axis = 1)
idx1 = np.where(pred_y == 0)[0]
pred_y_ = np.argmax(baseline_model1.predict(test_x), axis = 1)
idx1_ = np.where(pred_y_ == 0)[0]
tn_idx = set(idx).intersection(idx1)
tn_idx = set(tn_idx).intersection(idx1_)
abnormal_test = test_x[list(tn_idx)]

# obtain true negative set of train set
idx2 = np.where(train_y == 1)[0]
pred_ty = baseline_model.predict(train_x).round()
pred_ty = np.argmax(baseline_model.predict(train_x), axis = 1)
idx3 = np.where(pred_ty == 1)[0]
tp_idx = set(idx2).intersection(idx3)
normal_test = train_x[list(tp_idx)]

# initialize the evaluator
evaluator = Evaluator(train_x, normal_test)

61/61 [==============================] - 0s 833us/step


In [8]:
thyroid_update_path = 'thyroid_sns_update.json'
run(thyroid_update_path)

[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
